In [16]:
import keras
from keras.preprocessing import sequence
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed, Merge, Input, concatenate
#from keras.utils.visualize_util import plot, to_graph
#import theano.d3viz as d3v
import word2vec_utils as w2v
import data
import numpy as np
from data_utils import split_dataset 
from chat_constants import *

In [2]:
w2v_model = w2v.initialize()

In [4]:
assert( len(w2v_model.vocab) == 3000000 )


In [20]:
LSTM_DROPOUT = 0.15
A1, B, A2 = data.load_data()
def get_sets_of_data(size=5000):
    A1, B, A2 = data.load_data()
    All_data = np.array([A1,B,A2])
    num_sections = len(A1)//size
    sets = []
    for i in range(num_sections):
        sets.append(All_data[:,i*size:(i+1)*size])
    return(sets)
sets = get_sets_of_data()
adam = keras.optimizers.Adam(lr = 0.025)


In [35]:
### A Layer Pre Train ### 
A_Input = Input(shape =(MAX_SENT_LENGTH,EMBED_DIM))
A_layer1 = LSTM(EMBED_DIM,return_sequences=True, name = "layer_a", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
a_pretrain = Model( inputs = [A_Input], outputs = [ A_layer1(A_Input)])
a_pretrain.load_weights('AmergeB_len30_301dim_w2v_in_and_out.h5',by_name = True)
a_pretrain.compile( optimizer='rmsprop', loss = 'cosine_proximity',metrics = ['accuracy'] )


In [36]:
a1,b1,a2 = sets[3]
A1_train,B_train,A2_train = w2v.get_training_data(a1,b,a2)
a_pretrain.load_weights('a_pretrain.h5',by_name=True)
a_pretrain.fit([A1_train],A2_train, batch_size = 200, epochs = 5)
a_pretrain.save('a_pretrain.h5')

Epoch 1/5
5000/5000 [==============================] - 24s - loss: -0.0021 - acc: 0.4847    
Epoch 2/5
5000/5000 [==============================] - 23s - loss: -0.0021 - acc: 0.4852    
Epoch 3/5
5000/5000 [==============================] - 24s - loss: -0.0021 - acc: 0.4849    
Epoch 4/5
5000/5000 [==============================] - 25s - loss: -0.0021 - acc: 0.4854    
Epoch 5/5
5000/5000 [==============================] - 22s - loss: -0.0021 - acc: 0.4855    


In [33]:
### A Layer Pre Train ### 
B_Input = Input(shape =(MAX_SENT_LENGTH,EMBED_DIM))
B_layer1 = LSTM(EMBED_DIM,return_sequences=True, name = "layer_b", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
b_pretrain = Model( inputs = [B_Input], outputs = [ B_layer1(B_Input)])
#b_pretrain.load_weights('AmergeB_len30_301dim_w2v_in_and_out.h5',by_name = True)
b_pretrain.compile( optimizer='rmsprop', loss = 'cosine_proximity',metrics = ['accuracy'] )


In [34]:
a1,b1,a2 = sets[4]
A1_train,B_train,A2_train = w2v.get_training_data(a1,b,a2)
b_pretrain.fit([B_train],A2_train, batch_size = 200, epochs = 5)
b_pretrain.save('b_pretrain.h5')

Epoch 1/5
5000/5000 [==============================] - 25s - loss: -0.0014 - acc: 0.4564    
Epoch 2/5
5000/5000 [==============================] - 22s - loss: -0.0020 - acc: 0.5000    
Epoch 3/5
5000/5000 [==============================] - 22s - loss: -0.0021 - acc: 0.5122    
Epoch 4/5
5000/5000 [==============================] - 26s - loss: -0.0021 - acc: 0.5210    
Epoch 5/5
5000/5000 [==============================] - 27s - loss: -0.0022 - acc: 0.5278    


(5000, 30, 301)

In [39]:
A_Input = Input(shape =(MAX_SENT_LENGTH,EMBED_DIM))
B_Input = Input(shape =(MAX_SENT_LENGTH,EMBED_DIM))

A_layer1 = LSTM(EMBED_DIM,return_sequences=True, name = "layer_a", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
B_layer1 = LSTM(EMBED_DIM,return_sequences=True, name = "layer_b", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
A_layer1.trainable = False
B_layer1.trainable = False
merge    = concatenate( [A_layer1(A_Input),B_layer1(B_Input)])

AB_layer1 = LSTM(EMBED_DIM,return_sequences=True, name = "layer_ab1", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
AB_layer2 = LSTM(EMBED_DIM,return_sequences=True, name = "layer_ab2", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)

AB_output = AB_layer2(AB_layer1(merge))

chat_model = Model(inputs = [ A_Input,B_Input], outputs = [AB_output])

chat_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_16 (InputLayer)            (None, 30, 301)       0                                            
____________________________________________________________________________________________________
input_17 (InputLayer)            (None, 30, 301)       0                                            
____________________________________________________________________________________________________
layer_a (LSTM)                   (None, 30, 301)       726012                                       
____________________________________________________________________________________________________
layer_b (LSTM)                   (None, 30, 301)       726012                                       
___________________________________________________________________________________________

In [40]:
chat_model.load_weights('a_pretrain.h5',by_name=True)
chat_model.load_weights('b_pretrain.h5',by_name=True)
chat_model.load_weights('AmergeB_len30_30dim_w2v_in_and_out.h5')

In [ ]:
chat_model.compile( optimizer='rmsprop',loss = 'cosine_proximity',metrics = ['accuracy'])
for a1,b,a2 in sets:
    A1_train,B_train,A2_train = w2v.get_training_data(a1,b,a2)
    chat_model.fit([A1_train,B_train], A2_train,
          batch_size=200, epochs=5,
          validation_split = 0.05 )
    chat_model.save('AmergeB_len30_301dim_w2v_in_and_out.h5')

In [43]:
i = 1
A1_test,B_test,A2_test = w2v.get_training_data(A1[i:i+1],B[i:i+1],A2[i:i+1])
print(A1[i])
print(B[i])
print(A2[i])

predicted = chat_model.predict([A1_test,B_test])
predicted_words = w2v.unvectorize_sentence( predicted[0] )
print( predicted_words )

1 9 1 @ Well , I thought we'd start with pronunciation , if that's okay with you .
2 8 1 @ Not the hacking and gagging and spitting part . Please .
1 9 1 @ Okay . . . then how 'bout we try out some French cuisine . Saturday ? Night ?
('w2v word-score', u'1', 0.9904819130897522, '\n', 'uk word-score', 'at-arms', 0.25018321554559492)
('w2v word-score', u'9', 0.9829464554786682, '\n', 'uk word-score', 'at-arms', 0.24525209555474739)
('w2v word-score', u'1', 0.9902269840240479, '\n', 'uk word-score', 'at-arms', 0.24893820822454898)
('w2v word-score', u'@', 0.9984010457992554, '\n', 'uk word-score', 'aboard--you', 0.20060584535994638)
('w2v word-score', u'You', 0.7782272100448608, '\n', 'uk word-score', 'thirty--', 0.20711186238067558)
('w2v word-score', u'do', 0.7360661029815674, '\n', 'uk word-score', "carl's", 0.21274558816409631)
('w2v word-score', u'do', 0.7273008227348328, '\n', 'uk word-score', '.', 0.23543978289334419)
('w2v word-score', u'do', 0.7091096639633179, '\n', 'uk word-sco